In [1]:
import numpy as np
from astropy.io import ascii
from scipy import interpolate
from bit_tools.mission_tools import Mission
from mag_converter import *

In [48]:
magcut = 26.0
DIRECTORY = '/mnt/44868D0F868D02A2/Users/veome/Google Drive/Academics/Senior Thesis/Code/Catalog/'

In [51]:
cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)

/home/veome/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ConversionWarning: Some errors were detected !
    Line #2617 (got 10 columns instead of 11)
  """Entry point for launching an IPython kernel.


In [6]:
# Get HST F814W Band from bit_tools
hst = Mission("hst")
band = hst.get_bandpass("f814w")
f814w_band_response = band.r_lam
# Change units from nm to Angstrom
x = band.get_wavelengths() 
f814w = interpolate.interp1d(x*10, f814w_band_response(x), fill_value="extrapolate")

In [7]:
# Get SuperBIT Shape Band from bit_tools
bit = Mission("superbit")
r_shape = bit.get_bandpass("shape").r_lam
# Change units from nm to Angstrom
# x = bit.get_bandpass("shape").get_wavelengths()
x = np.arange(300, 1200, 1)
sbit_shape = interpolate.interp1d(x*10, r_shape(x), fill_value="extrapolate")

In [8]:
# Elliptical Template
elliptical_template = np.genfromtxt('galaxy_spectra_kinney96/elliptical_template.ascii', usecols=range(2), invalid_raise=False)

In [30]:
# Starburst Template
starburst_all = [[], [], [], [], [], []]
for i in range(1, 7):
    starburst_all[i-1] = np.genfromtxt('galaxy_spectra_kinney96/starb%i_template.ascii' %(i), usecols=range(2), invalid_raise=False)

starburst_template = np.mean( np.array([starburst_all[0], starburst_all[1], starburst_all[2], starburst_all[3], starburst_all[4], starburst_all[5]]), axis=0 )

In [61]:
# Initialization
cat = cosmos[cosmos[:,2] > 0]
cat = cat[cat[:,5] <= magcut]
length = cat.shape[0]
width = cat.shape[1]

print("Original Catalog Size: %i"%cosmos.shape[0])
print("New Catalog Size: %i"%length)

Original Catalog Size: 697594
New Catalog Size: 389181


In [62]:
# Mag conversion
mag_obs = cat[:,5]
z = cat[:,2]
super_e_mag = np.zeros(length)
super_starb_mag = np.zeros(length)

for i in range(length):
    if (i%(10000) == 0):
        print("Object %i of %i (%i\%)" %(i, length, (i//length)*100))
        
    super_e_mag[i], super_starb_mag[i] = stmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template)    
    
print("Conversion complete!")

Object 1 of 389181
Object 10001 of 389181
Object 20001 of 389181
Object 30001 of 389181
Object 40001 of 389181
Object 50001 of 389181
Object 60001 of 389181
Object 70001 of 389181
Object 80001 of 389181
Object 90001 of 389181
Object 100001 of 389181
Object 110001 of 389181
Object 120001 of 389181
Object 130001 of 389181
Object 140001 of 389181
Object 150001 of 389181
Object 160001 of 389181
Object 170001 of 389181
Object 180001 of 389181
Object 190001 of 389181
Object 200001 of 389181
Object 210001 of 389181
Object 220001 of 389181
Object 230001 of 389181
Object 240001 of 389181
Object 250001 of 389181
Object 260001 of 389181
Object 270001 of 389181
Object 280001 of 389181
Object 290001 of 389181
Object 300001 of 389181
Object 310001 of 389181
Object 320001 of 389181
Object 330001 of 389181
Object 340001 of 389181
Object 350001 of 389181
Object 360001 of 389181
Object 370001 of 389181
Object 380001 of 389181
Conversion complete!


In [63]:
all_data = np.zeros((length,width+2))
all_data[:,0:width] = cat
all_data[:,width] =  np.around(super_e_mag, decimals=5)
all_data[:,width+1] = np.around(super_starb_mag, decimals=5)

In [64]:
headers = np.array(['# RA [deg]', 'Dec [deg]', 'Redshift', 'gamma1', 'gamma2', 'Magnitude', 'Weight', 'Size', 'UncorSize', 'FWHM', 'SNR_HST', 'SuperBIT E Mag', 'SuperBIT Starburst Mag'])

In [65]:
ascii.write(all_data, 'superbit_cosmos1361_shape_mag26.gcat_ascii', delimiter=' ', format='fixed_width', names=headers, overwrite=True)